<a href="https://colab.research.google.com/github/Hemanthk1099/final_year_project/blob/main/final_year_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyfpgrowth

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.cluster import DBSCAN
from sklearn.metrics import jaccard_score
from pandas import DataFrame
import pyfpgrowth
import datetime
# from fp_growth import find_frequent_itemsets
# from mlxtend.frequent_patterns import fpgrowth

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/final_year_project/Market_Basket_Optimisation.csv')

In [ ]:
data.head()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_array = np.array(data)

In [ ]:
data_array[0:5]

array([['burgers', 'meatballs', 'eggs', nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['chutney', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan],
       ['turkey', 'avocado', nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['mineral water', 'milk', 'energy bar', 'whole wheat rice',
        'green tea', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan],
       ['low fat yogurt', nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=object)

In [ ]:
def preprocess1(data_array):
  org_data = dict()
  org_data_list = list()
  for test_list in data_array:
    res = []
    for val in test_list:
        if str(val) != 'nan':
            res.append(val)
    res.sort()
    org_data_list.append(res)
    org_data_list.sort(key = len , reverse=True)
    i = 0
    for trans in org_data_list:
      org_data[i] = trans
      i += 1
  return org_data , org_data_list

In [ ]:
# org_data_list list of transactions sorted in descending order
# org_data dictionary of transactions - key is transaction id
org_data , org_data_list = preprocess1(data_array)

In [ ]:
#find freq table
def find_freq_table(transactions):
  freq_table = dict()
  for transaction in transactions:
    for item in transaction:
      if item not in freq_table.keys():
        freq_table[item] = 1
      else:
        freq_table[item] += 1
  freq_table = {k: v for k, v in sorted(freq_table.items(), key=lambda item: item[1])}
  return freq_table

In [ ]:
freq_table = find_freq_table(org_data.values())

In [ ]:
# generate item_id and id_item dict
def id_item_dict(items):
  i = 0
  item_id = dict()
  id_item = dict()
  for item in items:
    item_id[item] = i
    id_item[i] = item
    i += 1
  return item_id , id_item

In [ ]:
item_id, id_item = id_item_dict(freq_table.keys())

In [ ]:
## find one hot encoding of the database
def find_one_hot(data: dict, item_id: dict):
  one_hot_dict = dict()
  for key in data.keys():
    one_hot = np.zeros(len(item_id))
    for item in data[key]:
      one_hot[item_id[item]] = 1
    one_hot_dict[key] = one_hot
  return one_hot_dict

In [ ]:
one_hot_dict = find_one_hot(org_data,item_id)

In [ ]:
def jaccard_index(s1, s2) :
      
    # Sizes of both the sets 
    size_s1 = len(s1); 
    size_s2 = len(s2); 
  
    # Get the intersection set 
    intersect = set(s1).intersection(set(s2)); 
  
    # Size of the intersection set 
    size_in = len(intersect); 
  
    # Calculate the Jaccard index 
    # using the formula 
    jaccard_in = size_in  / (size_s1 + size_s2 - size_in); 
  
    # Return the Jaccard index 
    return 1-jaccard_in; 

In [ ]:
# set k no of clusters = no of unique items
k = len(item_id)
k

120

In [ ]:
# compare function for calculation of centroids
def compare(a , b):
  for i in range(0,len(b)):
    if b[i] == 1 and a[i] == 0:
      return False
  return True

In [ ]:
# items is one-hot-encoded-transactions
def find_k_centroids(items,k):
  centroids = dict()
  for key in items.keys():
    if len(centroids) <k:
      check = False
      for centroid_key in centroids.keys():
        if compare(centroids[centroid_key],items[key]):
          check = True
          break
      if not check:
        centroids[key] = items[key]
  return centroids

In [ ]:
centroids = find_k_centroids(one_hot_dict,k)

In [ ]:
def get_centroids_with_id(centroids):
  centroids_with_id = dict()
  i =0
  for centroid in centroids.values():
    centroids_with_id[i] = centroid
    i += 1
  return centroids_with_id

In [ ]:
centroids_with_id = get_centroids_with_id(centroids)

In [ ]:
# returns a dict with key = transaction's key and value = cluster id which the item belongs to
def cluster(centroids, items):
  clusters_labels = dict()
  for item_key in items.keys():
    clusters_labels[item_key] = 0
  for item_key in items.keys():
    # closest_cluster = 0
    min_dist = 1
    for centroid_key in centroids.keys():
      dist = jaccard_index(org_data_list[centroid_key],org_data_list[item_key])
      if dist < min_dist:
        min_dist = dist
        clusters_labels[item_key] = centroid_key
  return clusters_labels

In [ ]:
# clusters -dict of item_id-cluster_id , data - dict of item_id-transaction
def prune_database(clusters, data, min_sup, number_of_clusters):
  # count - number of items of a cluster - cluster_id - corresponding count 
  count = dict()
  for i in range(0,number_of_clusters):
    count[i] = 0
  for i in clusters.keys():
    count[clusters[i]] += 1

  cluster_keys = set(count.keys())
  # deleting cluster_ids with count < min_sup
  for i in cluster_keys:
      if count[i] < min_sup:
        count.pop(i) 
  
  # list
  reduced_data = list()
  
  # set of cluster_ids after pruning
  cluster_keys = set(count.keys())
  
  for i in data.keys():
    if clusters[i] in cluster_keys:
      reduced_data.append(data[i])
    
  return reduced_data

In [ ]:
# data = dict
def freq_pattern_with_clustering(centroids_with_id ,one_hot_dict,data,min_sup_count, k ):

  t1 = datetime.datetime.now()
  clusters = cluster(centroids_with_id,one_hot_dict)
  pruning_parameter = 0.05
  reduced_data = prune_database(clusters,data,int(min_sup_count*pruning_parameter),k)
  t2 = datetime.datetime.now()
  print("The dataset size reduced from "+str(len(data))+" to "+str(len(reduced_data)))
  print("time taken for clustering and pruning: "+ str(t2-t1))

  r = len(data) - len(reduced_data)

  t2 = datetime.datetime.now()
  min_support_count_for_reduced_dataset = int(min_sup_count*len(reduced_data)/len(data))
  freq_itemsets = pyfpgrowth.find_frequent_patterns(reduced_data, min_support_count_for_reduced_dataset)
  t3 = datetime.datetime.now()

  print("time taken for fpgrowth with clustering: "+ str(t3-t2))
  # print(len(freq_itemsets))
  # print(freq_itemsets)
  return r

In [ ]:
# data - array / list , min_sup -int
def frequent_pattern_generation (data , min_sup_count):
  a = datetime.datetime.now()
  freq_itemsets = pyfpgrowth.find_frequent_patterns(data, min_sup_count)
  b = datetime.datetime.now()
  print("Time taken by fpgrowth without clustering :"+ str(b-a))
  # print(len(freq_itemsets))
  # print(freq_itemsets)
  return freq_itemsets

In [ ]:
for min_sup_count in range(100,1500,100):
  print("for min_sup = "+str(min_sup_count))
  r = freq_pattern_with_clustering(centroids_with_id, one_hot_dict, org_data, min_sup_count, k)
  frequent_pattern_generation(org_data.values() , min_sup_count)
  print("\n")

for min_sup = 100
The dataset size reduced from 7500 to 7500
time taken for clustering and pruning: 0:00:01.622844
time taken for fpgrowth with clustering: 0:00:00.492611
Time taken by fpgrowth without clustering :0:00:00.369698


for min_sup = 200
The dataset size reduced from 7500 to 7450
time taken for clustering and pruning: 0:00:01.612014
time taken for fpgrowth with clustering: 0:00:00.245513
Time taken by fpgrowth without clustering :0:00:00.270924


for min_sup = 300
The dataset size reduced from 7500 to 7338
time taken for clustering and pruning: 0:00:01.672205
time taken for fpgrowth with clustering: 0:00:00.341644
Time taken by fpgrowth without clustering :0:00:00.191653


for min_sup = 400
The dataset size reduced from 7500 to 7127
time taken for clustering and pruning: 0:00:01.604644
time taken for fpgrowth with clustering: 0:00:00.122998
Time taken by fpgrowth without clustering :0:00:00.133322


for min_sup = 500
The dataset size reduced from 7500 to 6832
time taken for 